<a href="https://colab.research.google.com/github/BachiLi/A-Tour-of-Computer-Animation/blob/main/A_Tour_of_Computer_Animation_1_Newtonian_Mechanics_and_Forward_Euler_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Animation is about the relation between spatial properties and time. We represent this relation using a function over time. For example, $x(t) = t$ is an animation where the position of an object depends linearly on time. 

In [1]:
# Our animation function
def x(t):
    return t

Let's try to visualize this animation.

In [2]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

fps = 20

def visualize(x):
    fig = plt.figure()
    ax = plt.axes(xlim=(0, 3), ylim=(-2, 2))
    point, = ax.plot([], [], 'g.', ms=20)
    plt.axis('off')

    def animate(i):
        t = i / fps
        point.set_data([x(t)], [0])
        return point,

    plt.close()
    return animation.FuncAnimation(fig, animate, frames=60, interval=fps, blit=True)
anim = visualize(x)
HTML(anim.to_html5_video())

We've created our first animation! The whole field of computer animation is about solving for the relation between spatial properties and time. We can let users or animators specify this relation, but this can often be tedious. Therefore we often rely on physics. For most computer animation applications, we apply principles from [classical mechanics](https://en.wikipedia.org/wiki/Classical_mechanics), which studies the motion of "reasonably large objects", such as a ball, a hair strand, a piece of cloth, a raindrop, a missle, or a planet.

A crucial assumption of classical mechanics, called the Newton's principle of determinacy, is that all motions of a system are uniquely determined by their initial positions and initial velocities. As a result, there is a function $F$ such that:

$\ddot{x} = F(x, \dot{x}, t)$

Here, $x=x(t)$ is the position of an object. $\dot{x} = \frac{dx}{dt}$ is the velocity of the object. $\ddot{x} = \frac{d\dot{x}}{dt}$ is the acceleration. We call this form the "Newtonian mechanics". In fact, the function $F$ is what usually called the *force* acting on the object. The equation above is a different way of writing the famous equaiton $F=ma$.

As an example, it has been shown experimentally that for a ball falling to the earth:

$\ddot{x} = -9.8$

The equation above is called an (ordinary) differential equation. Given the differential equation, our goal is to solve for the motion of the ball $x(t)$. Often we are also given the initial states $x(0)$ and $\dot{x}(0)$. This is called an [*initial value problem*](https://en.wikipedia.org/wiki/Initial_value_problem). In this particular case, we can solve for $x$ exactly by integrating both sides:

$x(t) = -\frac{9.8}{2} t^2 + \dot{x}(0)t + x(0)$

Let's try to visualize this! We will give the ball an initial up velocity and let the gravity pulls the ball.

In [3]:
x0 = 10.0
v0 = 4.0
def x(t):
    return -9.8/2 * t * t + v0 * t + x0

def visualize(x):
    fig = plt.figure()
    ax = plt.axes(xlim=(-2, 2), ylim=(-12, 12))
    point, = ax.plot([0], [0], 'g.', ms=20)
    plt.axis('off')

    def animate(i):
        t = i/fps
        point.set_data([0], [x(t)])
        return point,

    plt.close()
    return animation.FuncAnimation(fig, animate, frames=100, interval=fps, blit=True)

anim = visualize(x)
HTML(anim.to_html5_video())

The ball above does not collide with ground. To take that into consideration, we need to add a *constraint*. We assume the ground is at $x=0$, then the collision with the ground introduces a constraint:

$x(t') = 0 \rightarrow \dot{x}_{+}(t') = -\dot{x}_{-}(t')$

where $\dot{x}_{+}(t') = \lim_{t \rightarrow t'^{+}} \dot{x}(t)$ is the right limit and $\dot{x}_{-}(t')$ is the left limit. This can be derived from the momentum and energy conservation. Unfortunately, by adding this constraint, the velocity $\dot{x}$ becomes not differentiable at the collision point $t'$. This is fine: since $\dot{x}$ is still differentiable almost everywhere, we can still integrate its derivative $\ddot{x}$ to solve our positions (in particular, the acceleration $\ddot{x}$ here becomes a [Dirac delta function](https://math.mit.edu/~stevenj/18.303/delta-notes.pdf) at $t'$).

Another unfortunateness is that it's now much more difficult to derive the closed-form solution. Typically, we have to rely on numerical approximated solutions. How do we solve an ordinary differential equation numerically? Firstly, the ODE above is a second-order ODE -- this means that it involves second derivatives. This makes things complicated. Fortunately we can always convert a second-order ODE into a first order ODE system. In this case, it involves two variables $x$ and $v=\dot{x}$:

$
\begin{split}
\dot{x}(t) &= v(t) \\
\dot{v}(t) &= -9.8
\end{split}
$

Next, we use the definition of the derivatives to approximate the ODE system above:

$
\begin{split}
x(t + \Delta_t) &= x(t) + \Delta_t v \\
v(t + \Delta_t) &= v(t) - 9.8 \Delta_t
\end{split}
$

The approximation above is exact when the time step $\Delta_t$ goes to 0 in the limit. In practice, we will choose a finite time step. This leads to approximation error which is usually called the *discretization error*. There are many different ways to approximate ODEs numerically with different trade-offs such as performance, accuracy, and stability. These methods are called the [*integrators*](https://en.wikipedia.org/wiki/Numerical_methods_for_ordinary_differential_equations). We will introduce some of them in the future. For now we will use the simplest method above, usually called the [*forward Euler method*](https://en.wikipedia.org/wiki/Euler_method).

The approximation still does not take the collision contraint into account. We resolve the constraint by checking if $x(t + \Delta_t) \leq 0$. If so, we will find the time step such that the ball hits exactly at the floor. Then we will flip the velocity. In general, solving for collisions between objects efficiently and correctly is a hard problem, and is an active area of research. We will discuss this in the future.

Let's put this into code!

In [4]:
x0 = 10.0
v0 = 4.0
ts = 0.001
# Implement the forward Euler method to integrate for time t given current x & v
def forward_euler_method(t, x, v):
    ct = 0
    while True:
        nx = x + ts * v
        nv = v - 9.8 * ts
        if nx > 0:
            x = nx
            v = nv
            ct += ts
        else:
            # Collision event, search for t' such that x + t' * v = 0
            tp = -x/v
            x = 0.0
            v = -v
            ct += tp
        if ct >= t:
            break
    return ct, x, v

def visualize(ode_solver):
    fig = plt.figure()
    ax = plt.axes(xlim=(-2, 2), ylim=(0, 12))
    point, = ax.plot([0], [0], 'g.', ms=20)
    plt.axis('off')

    t = 0
    x = x0
    v = v0
    def animate(i):
        nonlocal t, x, v
        dt, x, v = ode_solver(1.0/fps, x, v)
        t += dt
        point.set_data([0], [x])
        return point,

    plt.close()
    return animation.FuncAnimation(fig, animate, frames=300, interval=fps, blit=True)

anim = visualize(forward_euler_method)
HTML(anim.to_html5_video())

These examples are simple and maybe even trivial, but they demonstrate the typical process of a computer animation algorithm:
1.   Model the motion using a differential equation or its equivalent.
2.   Specify the constraints and initial/boundary conditions.
3.   Discretize the continuous equations, and solve for the states at a specific time.

Most of our future examples will follow this structure.


We will close this first tour by having an example with multiple 2D particles. We will solve the so-called N-body problem. Newton observed that objects pull each other with forces. The position $x_i$ of a particle $i$ is governed by:

$\ddot{x}_i = \sum_{j \neq i} \frac{G}{\| x_j - x_i \|^2} \frac{x_j - x_i}{\| x_j - x_i \|} $

where $G$ is a constant -- we will just assume it is $1$. We assume all the particles have the same mass. Using the exact same methodology as before, we can solve for the n-body problem using follow code:

In [5]:
import numpy as np
x0 = np.array([[1.0, -1.0], [0.0,  0.0]])
v0 = np.array([[0.0,  0.0], [0.5, -0.5]])
ts = 0.001
def forward_euler_method(t, x, v):
    ct = 0
    while True:
        nx = x + ts * v
        nv = v
        for i in range(x.shape[1]):
            for j in range(x.shape[1]):
              if i != j:
                diff = x[:, j] - x[:, i]
                nv[:, i] += ts * 1.0 * diff / np.power(np.linalg.norm(diff), 3.0)
        x = nx
        v = nv
        ct += ts
        if ct >= t:
            break
    return ct, x, v

def visualize(ode_solver):
    fig = plt.figure()
    ax = plt.axes(xlim=(-2, 2), ylim=(-2, 2))
    point, = ax.plot([], [], 'g.', ms=20)
    plt.axis('off')

    t = 0
    x = x0
    v = v0
    def animate(i):
        nonlocal t, x, v
        dt, x, v = ode_solver(1.0/fps, x, v)
        t += dt
        point.set_data(x[0, :], x[1, :])
        return point,

    plt.close()
    return animation.FuncAnimation(fig, animate, frames=1000, interval=fps, blit=True)

anim = visualize(forward_euler_method)
HTML(anim.to_html5_video())